## Using `NameIt`




<a name='nb-top' href='#'>Top</a>

### Using [github.com/dfinke/NameIt](https://github.com/dfinke/NameIt)

- part1: [Builtin Template Strings](#nb-part1)
- part2: [Template type Summary Table](#nb-part2)
- part3: [New-NameItTemplate](#nb-part3)

This first block is for helper functions.
You can mostly ignore it,The important code comes after. It's written to be short, not written to be "good" (ie: the helper functions are not best practice)

In [ ]:
function nb.JoinCsv { 
    [Alias('Csv', 'UL')]
    [cmdletbinding()]
    param(
        <# prefix string #>     [Alias('op')][Parameter(Position=0)]$Prefix       = '',
        <# suffix string #>     [Alias('os')][Parameter(Position=1)]$Suffix       = '',
        <# suffix string #>     [Alias('Sep')][Parameter()]$Separator             = ', ',
        <# suffix property #>   [Alias('Prop', 'P')][Parameter()]$PropertyName    = $Null,
        <# Actual Pipe #>[Parameter(ValueFromPipeline, Mandatory)][object[]]$InputObject,
        [switch]$Sort, [switch]$Unique, [int]$Pad = 0
        # [switch]$Qoute
    )
    begin {
        $items = [System.Collections.Generic.list[object]]::new()
    }
    process {
        foreach($cur in $InputObject) { $Items.add($cur)}
    } 
    end {
        $isUL = $PSCmdlet.MyInvocation.InvocationName -eq 'UL'

        $padding = "`n" * $Pad -join ''
        $Prefix = $Padding + $Prefix
        $Suffix = $Suffix + $Padding

        
        if($False -and 'original UL' -and $IsUL) {             
                $Prefix = $Prefix + "`n - "
                $Suffix = "`n - " + $Suffix
                $Separator = "`n - "
        }
        if($IsUL) {             
            if( (-not $Separator) -or $Separator -eq ', ') {
                $Separator = ' - '    
            }
            $Prefix = $Prefix + "`n${Separator}"
            $Suffix = "`n" + $Suffix
            # $Suffix = "`n${Separator}" + $Suffix # was
            $Separator = "`n${Separator}"
        }
        
        $splat = @{  'sep' = $Separator ; 'op' =  $Prefix ; 'os' = $Suffix }
        $IsRawText = $items[0] -is 'string'
        write-debug $IsRawText
        
        $filtered = if($Unique) { $Items | Sort -Unique }
        elseif($Sort) { $Items | sort }
        else { $items } 

        if($PropertyName -and (! $IsRawText)) { 
            $filtered | Join-String @splat -Prop $PropertyName
            # $Items | Join-String -sep ', ' -op:$Prefix -os $Suffix -Prop $PropertyName
        } else {
            $filtered | Join-String @splat
            # $Items | Join-String -sep ', ' -op:$Prefix -os $Suffix
        }
    }
}

('last executed: {0}' -f @((get-date).tostring('u')))
Import-Module NameIt -ea stop -passthru | Ul -Prop { $_.Name, $_.Version } -op 'modules = '
Get-Command -m NameIT | ul -prop Name -op 'NameIt Commands = '

last executed: 2022-07-06 12:48:32Z
modules = 
 - NameIt 2.3.2

NameIt Commands = 
 - Invoke-Generate
 - New-NameItTemplate



In [ ]:
$Templates = @(
    '[person], [color]'
    '[address]'
    '[synonym cafe]_[syllable][syllable]'
    '####'
    '??-###-####-?##'
); $Max = $Templates | Measure-object Length -max | % Maximum
$Templates | %{ 
    foreach($i in 0..3) { 
    @(
        $_.PadLeft($max)
        ig $_
    ) -join ' = '
    }
}


                  [person], [color] = Ernest Richardson, beige
                  [person], [color] = Helen Mcneil, crimson
                  [person], [color] = Blaine Forbes, red
                  [person], [color] = Calvin Barton, cyan
                          [address] = 59145 Tifu Lck
                          [address] = 98871 Vexeyimuyax Blf
                          [address] = 498 Etniyuzheuv Run
                          [address] = 12541 Jege Glen
[synonym cafe]_[syllable][syllable] = restaurant_ihot
[synonym cafe]_[syllable][syllable] = restaurant_ayap
[synonym cafe]_[syllable][syllable] = restaurant_umos
[synonym cafe]_[syllable][syllable] = coffeebar_xaoc
                               #### = 4628
                               #### = 9883
                               #### = 7582
                               #### = 6000
                    ??-###-####-?## = qs-012-2711-m70
                    ??-###-####-?## = yl-119-2603-x00
                    ??-###-####-?## = iq-5

<a href='#nb-top'>Go Top</a>
## <a name="nb-part1">Built-in Template Strings</a> 

In [ ]:

$Templates | Foreach-Object { 
    $_ | Csv 'Template = "' -os '"'
    @(foreach($i in 0..3) { 
        Invoke-Generate $_     
    }) | UL -sep ' '
}

Template = "[person], [color]"

 Jamar Macdonald, white
 Ronin Mckay, gray
 Jackson Reyes, beige
 Cullen Whitney, amber

Template = "[address]"

 53951 Zileagcix Sqrs
 2637 Etev Forks
 9797 Eredgedtic Forges
 56775 Vorun Field

Template = "[synonym cafe]_[syllable][syllable]"

 coffeehouse_fiol
 coffeehouse_wumjab
 eatingplace_mulo
 eatingplace_kompo

Template = "####"

 6349
 8032
 3089
 9603

Template = "??-###-####-?##"

 kj-287-4981-a19
 mu-008-4413-h05
 ab-967-9481-z23
 xf-427-8246-n96



<a href='#nb-top'>Go Top</a>
## <a name="nb-part2">Template Types Reference</a> 
 

| Template                | Description                                                                                        |
| ----------------------- | -------------------------------------------------------------------------------------------------- |
| `[alpha]`               | selects a random character (constrained by the -Alphabet parameter).                               |
| `[numeric]`             | selects a random numeric (constrained by the -Numbers parameter).                                  |
| `[vowel]`               | selects a vowel from a, e, i, o or u.                                                              |
| `[phoneticVowel]`       | selects a vowel sound, for example ou.                                                             |
| `[consonant]`           | selects a consonant from the entire alphabet.                                                      |
| `[syllable]`            | generates (usually) a pronouncable single syllable.                                                |
| `[synonym word]`        | finds a synonym to match the provided word.                                                        |
| `[person]`              | generate random name of female or male based on provided culture like <FirstName><Space><LastName> |
| `[person female]`       | generate random name of female based on provided culture like <FirstName><Space><LastName>.        |
| `[person female first]` |
| `[person female last]`  |
| `[person male]`         | generate random name of male based on provided culture like <FirstName><Space><LastName>.          |
| `[person male first]`   | .                                                                                                  |
| `[person male last]`    | .                                                                                                  |
| `[person both first]`   | .                                                                                                  |
| `[person both last]`    | .                                                                                                  |
| `[address]`             | generate a random street address. Formatting is biased to US currently.                            |
| `[space]`               | inserts a literal space. Spaces are striped from the templates string by default.                  |
 

In [ ]:

#from the docs
Invoke-Generate -alpha 'abc' -count 3 | ul


Invoke-Generate -count 5 "[synonym cafe]_[syllable][syllable]" | csv -op 'syllable: ' -pad 1 -sep "`n- "


 - baacabbb
 - cacabaca
 - acbcaabb


syllable: coffeebar_wotyob
- eatingplace_tabfi
- eatingplace_obiq
- restaurant_ruhat
- eatingplace_avpe



In [ ]:
$Templates =  @('ThisQuarter', 'q1', 'q3', 'q3', 'q4', 'Today', 'Tomorrow', 'Yesterday', 'February', 'April', 'October')
$Templates | %{ 
    $_ | Csv 'Template = "' -os '"'
    @(foreach($i in 0..1) { 
        Invoke-Generate $_     
    }) | UL -sep ' '
}

Template = "ThisQuarter"

 9/25/2022
 9/11/2022

Template = "q1"

 1/23/2022
 3/2/2022

Template = "q3"

 9/24/2022
 8/12/2022

Template = "q3"

 8/9/2022
 7/23/2022

Template = "q4"

 10/29/2022
 11/17/2022

Template = "Today"

 7/6/2022
 7/6/2022

Template = "Tomorrow"

 7/7/2022
 7/7/2022

Template = "Yesterday"

 7/5/2022
 7/5/2022

Template = "February"

 2/20/2022
 2/8/2022

Template = "April"

 4/18/2022
 4/2/2022

Template = "October"

 10/28/2022
 10/19/2022



In [ ]:
$Kind = New-NameItTemplate -sb {
    [PSCustomObject]@{
        Company = ''      
        Name    = ''
        Age     = 0
        address = ''
        state   = ''
        zip     = ''
    }        

}
Invoke-Generate $Kind -Count 3 -AsPSObject | Format-Table



state    address              Name                 Company Age   zip
-----    -------              ----                 ------- ---   ---
Kansas   28172 Luyroci Common Finley Delgado       fdbbnk  81704 62701
Vermont  659 Yava Allee       Jefferson Cunningham xhvfuc  69772 85001
Arkansas 6304 Buow Dam        Mariam Moody         efkfzs  24713 36101



### Steps to use custom objects as templates

1. Create a `ScriptBlock` that returns a `[PSCustomObject]`
1. Call `Invoke-Generate $YourSB -Count 5`
1. Choose output type:
   1. raw text, 1 string per `-count` 
   2. or `-AsCsv` and `-AsTabDelimited`
   3. or a `[pscustomobject]`

> If a property name has has the value **name, zip, address, or state** the appropriate NameIT template is applied, **otherwise the type is inferred as numeric or alpha**.

```ps1
New-NameItTemplate {[PSCustomObject]@{Company="";Name=""}}
```

In [ ]:
'-AsPSObject' | csv -pad 1
Invoke-Generate $Kind -Count 2 -AsPSObject | Ft

'-AsCsv' | csv -pad 1
Invoke-Generate $Kind -Count 2 -AsCsv

'-AsTabDelimited' | csv -pad 1
Invoke-Generate $Kind -Count 2 -AsTabDelimited



-AsPSObject


state        address            Name           Company Age   zip
-----        -------            ----           ------- ---   ---
North Dakota 33485 Korof Trace  Tanner Baldwin exedcx  17005 83701
Maryland     25703 Eceq Centers Fisher Roy     vfvjnq  57190 59601


-AsCsv

"state","address","Name","Company","Age","zip"
"Delaware","2589 Xexurcarjas Brooks","Alana Briggs","pdyjjf","24572","59601"
"state","address","Name","Company","Age","zip"
"Nebraska","560 Rumawacpe Parkway","Lilly Decker","yzekgp","52429","43201"

-AsTabDelimited

"state"	"address"	"Name"	"Company"	"Age"	"zip"
"Maine"	"466 Kuhec Shl"	"Sandra Dougherty"	"uxglrc"	"94597"	"02901"
"state"	"address"	"Name"	"Company"	"Age"	"zip"
"Louisiana"	"98457 Xeulox Trnpk"	"Luciana Pratt"	"vhqkpo"	"9685"	"84101"


<a href='#nb-top'>Go Top</a>
<h4><a name='nb-bottom'>See More</a>

- other `.inpynb` here
- Works well with [dfinke/ImportExcel](https://github.com/dfinke/ImportExcel)
- repo: [dfinke/NameIt](https://github.com/dfinke/NameIt)
  

In [ ]:
# Testing output
'
object properties as a hash summary:
'
Invoke-Generate $Kind -Count 2
| Foreach-Object { 
    $_ -split "\r?\n" | csv } | Join-String -op '@{ ' -os '} '
'
Almost hashtable literals:
'
Invoke-Generate $Kind -Count 2
| Foreach-Object { 
    $_ -split "\r?\n"
    | Join-String -prop {$_ | Join-String -DoubleQuote -sep ' ;  ' } } | Join-String -op '@{ ' -os '} '



object properties as a hash summary:

@{ Company=steokt, Name=Avery Lowery, Age=39166, address=763 Vemvuk Causeway, state=Michigan, zip=43201Company=tnkmfv, Name=Oliver Blair, Age=74340, address=438 Aqxo Overpass, state=Rhode Island, zip=73101} 

Almost hashtable literals:

@{ "Company=lteonr""Name=Kelvin Bright""Age=66354""address=21228 Bofjoxemmid Stra""state=Virginia""zip=73301""Company=gjdqyy""Name=Haley Douglas""Age=33823""address=25062 Mikresnivic Creek""state=Montana""zip=39201"} 


In [ ]:
New-NameItTemplate {[PSCustomObject]@{Company="";Name=""}}

"round trip should not fail"
[PSCustomObject]@{ Company=$null; Name="" } 
| ConvertTo-Json | convertfrom-json
| ConvertTo-Json



Company=[alpha 6]
Name=[person]
round trip should not fail
{
  "Company": null,
  "Name": ""
}


In [ ]:
# Can json directly easily work?
$json = @'
{
    "Name": "",
    "State": "",
    "EmployeId": 0
}
'@
$Template = @{}

$Obj = $Json | ConvertFrom-Json

$Template.FromJson = New-NameItTemplate -sb { $obj }
$Template.FromCommand = New-NameItTemplate {
    [PSCustomObject]@{
        Name=""
        State=""
        EmployeeId=""
    }
}

'from a [PSCO]' | csv -pad 1
$Template.FromCommand

'from Json' | Csv -Pad 1
$Template.FromJson


from a [PSCO]

Name=[person]
State=[state]
EmployeeId=[alpha 6]

from Json

Name=[person]
State=[state]
EmployeId=[numeric 5]


<a href='#nb-top'>Go Top</a>
## <a name="nb-part3">New-NameItTemplate</a> 
 


In [ ]:
# City and County no longer output
Invoke-Generate '[city]' -Count 1       
Invoke-Generate '[city both]' -Count 1
Invoke-Generate '[city county]' -Count 1  -verbose -debug

city
city both
city county
